In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tinkoff.invest import Client, CandleInterval
from decimal import Decimal

In [2]:
token = 'ваш тиньков токен (api 2.0)'

In [3]:
def units_nano_convert(d):
    # https://github.com/Tinkoff/invest-python/issues/45
    nano = d['nano'] / Decimal("10e8")
    price = d['units'] + float(nano)
    return price

In [4]:
# Берём все тикеты IMOEX отсюда
table = pd.read_html('https://www.tinkoff.ru/invest/etfs/TMOS/structure/details/')      

df = table[0]
df = df.rename(columns={'Название': 'name'})
df = df[df['name'] != 'Денежные средстваВалюта']
df['name'] = df['name'].str[:-5]

In [5]:
with Client(token) as client:
    shares = client.instruments.shares()

all_stocks = pd.DataFrame(shares.instruments)

df = pd.merge(df, all_stocks, how='left', on='name', sort=True)

df['min_price_increment'] = df['min_price_increment'].apply(units_nano_convert) 

In [6]:
# Текущие цены акций
with Client(token) as client:
    last_prices = client.market_data.get_last_prices(figi=df['figi'].to_list())

last_prices = pd.DataFrame(last_prices.last_prices)

last_prices['price'] = last_prices['price'].apply(units_nano_convert)


df = last_prices.merge(df, on='figi')

df = df[['name', 'ticker', 'figi', 'lot', 'price', 'min_price_increment', 'currency']]

In [7]:
df = df[df['currency'] != 'usd']

In [8]:
# Считаем для стартового капитала в 10 тыс рублей
capital = 10000

df['lot_price'] = df['lot'] * df['price']
df['shares_available'] = capital // df['lot_price']
df['profit'] = df['shares_available']  * df['min_price_increment'] * df['lot']

In [9]:
df = df.sort_values(by=['profit'], ascending=False)
df.index = np.arange(1, len(df) + 1)

In [10]:
df

,name,ticker,figi,lot,price,min_price_increment,currency,lot_price,shares_available,profit
1,En+ Group,ENPG,BBG000RMWQD4,1,688.00000,0.500000,rub,688.00,14.0,7.00
2,Аэрофлот,AFLT,BBG004S683W7,10,30.62000,0.020000,rub,306.20,32.0,6.40
3,Petropavlovsk PLC,POGR,BBG00VPKLPX4,100,8.13500,0.005000,rub,813.50,12.0,6.00
4,Ozon Holdings PLC,OZON,BBG00Y91R9T3,1,1000.00000,0.500000,rub,1000.00,10.0,5.00
5,VK,VKCO,BBG00178PGX3,1,416.80000,0.200000,rub,416.80,23.0,4.60
6,ГДР X5 RetailGroup,FIVE,BBG00JXPFBN0,1,1107.50000,0.500000,rub,1107.50,9.0,4.50
7,Татнефть - привилегированные акции,TATNP,BBG004S68829,1,310.40000,0.100000,rub,310.40,32.0,3.20
8,Татнефть,TATN,BBG004RVFFC0,1,359.70000,0.100000,rub,359.70,27.0,2.70
9,Банк ВТБ,VTBR,BBG004730ZJ9,10000,0.01881,0.000005,rub,188.10,53.0,2.65
10,Fix Price Group,FIXP,BBG00ZHCX1X2,1,383.20000,0.100000,rub,383.20,26.0,2.60


In [11]:
# https://ru.wikipedia.org/wiki/Голубые_фишки
moexbc = ['GAZP', 'SBER', 'LKOH', 'MGNT', 'NVTK', 'SNGS', 'GMKN', 
          'ROSN', 'NLMK', 'TATN', 'MTSS', 'ALRS', 'YNDX', 'FIVE', 'CHMF']

In [12]:
df[df['ticker'].isin(moexbc)]

,name,ticker,figi,lot,price,min_price_increment,currency,lot_price,shares_available,profit
6,ГДР X5 RetailGroup,FIVE,BBG00JXPFBN0,1,1107.50,0.500,rub,1107.50,9.0,4.50
8,Татнефть,TATN,BBG004RVFFC0,1,359.70,0.100,rub,359.70,27.0,2.70
12,МТС,MTSS,BBG004S681W1,10,188.05,0.050,rub,1880.50,5.0,2.50
15,НОВАТЭК,NVTK,BBG00475KKY8,1,970.20,0.200,rub,970.20,10.0,2.00
16,Сургутнефтегаз,SNGS,BBG0047315D0,100,22.69,0.005,rub,2269.00,4.0,2.00
19,Северсталь,CHMF,BBG00475K6C3,1,1061.80,0.200,rub,1061.80,9.0,1.80
24,Роснефть,ROSN,BBG004731354,1,397.35,0.050,rub,397.35,25.0,1.25
27,АЛРОСА,ALRS,BBG004S68B31,10,80.11,0.010,rub,801.10,12.0,1.20
28,НЛМК,NLMK,BBG004S681B4,10,155.86,0.020,rub,1558.60,6.0,1.20
31,Магнит,MGNT,BBG004RVFCY3,1,4170.00,0.500,rub,4170.00,2.0,1.00


In [13]:
# Стоимость IMOEX, если купить всего по одному лоту
df['lot_price'].sum()

191062.7

In [14]:
# Стоимость MOEXBC, если купить всего по одному лоту
df[df['ticker'].isin(moexbc)]['lot_price'].sum()

43045.45

In [15]:
df.to_csv('data.csv')

In [16]:
# # таблица в виде изображения
# # https://stackoverflow.com/a/39358752/10418812

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import six

# df_print = df.copy()

# df_print = df_print[['name', 'ticker', 'min_price_increment', 'lot', 'price', 'shares_available', 'profit']]
# df_print['name'] = df_print['name'].apply(lambda x: x.replace('привилегированные', 'прив'))
# df_print['name'] = df_print['name'].apply(lambda x: x[:23])
# df_print['shares_available'] = df_print['shares_available'].astype(int)
# df_print['profit'] = df_print['profit'].round(5)
# cols = df_print.columns.to_list()
# df_print['N'] = df_print.index
# df_print = df_print[['N'] + cols]
# df_print.columns = ['Номер', 'Название', 'Тикет', 'Шаг цены', 'Кол-во акций в лоте', 'Цена', 'Кол-во акций', 'Прибыль за один шаг']

# def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
#                      header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
#                      bbox=[0, 0, 1, 1], header_columns=0,
#                      ax=None, **kwargs):
#     if ax is None:
#         size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
#         fig, ax = plt.subplots(figsize=size)
#         ax.axis('off')

#     mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

#     mpl_table.auto_set_font_size(False)
#     mpl_table.set_fontsize(font_size)

#     for k, cell in six.iteritems(mpl_table._cells):
#         cell.set_edgecolor(edge_color)
#         if k[0] == 0 or k[1] < header_columns:
#             cell.set_text_props(weight='bold', color='w')
#             cell.set_facecolor(header_color)
#         else:
#             cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
            
#     return ax

# render_mpl_table(df_print, header_columns=0, col_width=3.7)